Aggregation in SQL
-----

In [ ]:
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%sql sqlite:///world-db

Let's ask some SQL queries over the `world` dataset using aggregation. 
Our first query finds the average population of countries in Europe.

In [ ]:
%sql PRAGMA table_info(Country)

In [ ]:
%%sql
SELECT AVG(Population)
FROM Country
WHERE Continent = 'Europe';

The next query outputs the number of countries in Europe.

In [ ]:
%sql SELECT * FROM Country LIMIT 1;

In [ ]:
%%sql
SELECT COUNT(*) 
FROM Country
WHERE Continent = 'Europe';

Next, we want to compute the number of languages in the `CountryLanguage` table. Here we want to use `DISTINCT` inside the count, otherwise languages that are spoken in multiple countries will be counted multiple times!

In [12]:
%%sql
SELECT COUNT(DISTINCT Language)
FROM CountryLanguage;

 * sqlite:///world-db
Done.


COUNT(DISTINCT Language)
457


How can we find the name *and* population of the country with the max population in Europe? Notice that finding only the max population is easy, but if we want to output the country as well, it gets a bit tricky. We can write a nested query to solve this problem in an elegant way!

In [13]:
%sql SELECT * FROM CountryLanguage LIMIT 1;

 * sqlite:///world-db
Done.


CountryCode,Language,IsOfficial,Percentage
ABW,Dutch,T,5.3


In [16]:
%sql SELECT * FROM Country LIMIT 1;

 * sqlite:///world-db
Done.


Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
ABW,Aruba,North America,Caribbean,193.0,None,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129,AW


In [18]:
%%sql
SELECT Name, Population
FROM Country
WHERE Continent = 'Europe' AND Population = 
  (SELECT MAX(Population)
   FROM Country
   WHERE Continent = 'Europe');

 * sqlite:///world-db
Done.


Name,Population
Russian Federation,146934000


An alternative way is to use together `ORDER BY` and `LIMIT`. However, the two queries can sometimes give a different result. Can you think when this may happen?

In [19]:
%%sql
SELECT Name, Population
FROM Country
WHERE Continent = 'Europe'
ORDER BY Population DESC
LIMIT 1;

 * sqlite:///world-db
Done.


Name,Population
Russian Federation,146934000


We will see next how to use `GROUP BY`. The following SQL query computes the number of countries in every continent.

In [21]:
%%sql
SELECT Continent
FROM Country 
GROUP BY Continent ;

 * sqlite:///world-db
Done.


Continent
Africa
Antarctica
Asia
Europe
North America
Oceania
South America


We can combine `GROUP BY` with `ORDER BY` as well. The following SQL query finds out how many countries speak each language with percentage > 50% in decreasing order.

In [ ]:
%%sql
SELECT Language, COUNT(CountryCode) AS Num
FROM CountryLanguage
WHERE Percentage >= 50
GROUP BY Language
ORDER BY Num DESC ;

The `HAVING` clause allows us to express conditions over properties of *groups*, and not only individual tuples. `HAVING` always follows a `GROUP BY`. As an example, the following SQL query finds out the languages that are spoken in at least 3 different countries with percentage at least 50.

In [23]:
%%sql
SELECT Language
FROM CountryLanguage
WHERE Percentage >= 50 
GROUP BY Language
HAVING COUNT(CountryCode) > 2;

 * sqlite:///world-db
Done.


Language
Arabic
Creole English
Creole French
English
German
Serbo-Croatian
Spanish


**Exercise #1**: Write a query that outputs for each country the population of the most populated city, for countries with at least 10 cities.

In [ ]:
%%sql
SELECT C.NAME AS Country, MAX(T.Population) AS N
FROM City T, Country C
WHERE C.Code = T.CountryCode
GROUP BY C.Name
HAVING COUNT(T.ID) > 9
ORDER BY N DESC ;

In [ ]:
%sql PRAGMA table_info(Country);

In [ ]:
%sql PRAGMA table_info(City);

Let's see how the use of `HAVING` compares with the use of correlated queries. Suppose that we want to find the names of the countries that have more than 10 cities with population at least 1 million. Here is a nested query that computes that: 

In [ ]:
%%sql
SELECT C.name
FROM Country C
WHERE (SELECT COUNT(*) 
       FROM City
       WHERE City.CountryCode = C.Code
       AND City.Population >= 1000000) > 10; 

We can measure the execution time of the query using `%time`.

In [26]:
%time %sql SELECT C.name FROM Country C WHERE (SELECT COUNT(*) FROM City WHERE City.CountryCode=C.Code AND City.Population >= 1000000) > 10; 

 * sqlite:///world-db
Done.
CPU times: user 53.1 ms, sys: 2.56 ms, total: 55.6 ms
Wall time: 54.2 ms


**Exercise #2**: Write the above query using `HAVING` and time its execution. How much faster does it run?

In [25]:
%time %sql SELECT C.name FROM Country C, City T WHERE T.CountryCode=C.Code AND T.Population >= 1000000 GROUP BY C.name HAVING COUNT(*)> 10; 

 * sqlite:///world-db
Done.
CPU times: user 2.07 ms, sys: 1 ms, total: 3.07 ms
Wall time: 2.18 ms
